In [1]:
import json
import torch
from sentence_transformers import util
import os

In [2]:
big = "memoryalpha"
small = "stexpanded"

top_k = 10

mapping_file_big = "./data/triples_v2/" + big + "_mapping.json"
mapping_file_small = "./data/triples_v2/" + small + "_mapping.json"
exact_match_file = "./data/exact_match/" + big + "-" + small + ".json"

communities_big_file = "./results/communities_leiden/" + big + "/final.txt"
communities_small_file = "./results/communities_leiden/" + small + "/final.txt"

communitiy_embeddings_big_file = "./results/community_embeddings/" + big + ".json"
communitiy_embeddings_small_file = "./results/community_embeddings/" + small + ".json"

matched_communities_path = f"./results/matched_communities_leiden/top_{top_k}_pairs/"

In [3]:
with open(exact_match_file) as emf, open(mapping_file_big) as mfb, open(mapping_file_small) as mfs:
    mapping_big = json.load(mfb)
    mapping_small = json.load(mfs)
    exact_match = json.load(emf)

In [4]:
small_communities = []
big_communities = []

with open(communities_big_file) as cbf, open(communities_small_file) as csf:
    
    for line in csf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        small_communities.append(numbers_set)

    for line in cbf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        big_communities.append(numbers_set)

In [5]:
with open(communitiy_embeddings_big_file) as cebf, open(communitiy_embeddings_small_file) as cesf:
    community_embeddings_big = json.load(cebf)
    community_embeddings_big = {k: list(v.values()) for k, v in community_embeddings_big.items()}
    community_embeddings_small = json.load(cesf)
    community_embeddings_small = {k: list(v.values()) for k, v in community_embeddings_small.items()}

In [6]:
# community_embeddings_big['0']

In [7]:
embeddings_old_file = "./results/embeddings/stexpanded_lab_altlab_type_abs_comment_BAAI_bge-large-en-v1.5.json"
with open(embeddings_old_file) as eof:
    embeddings_old = json.load(eof)

In [8]:
# embeddings_old['0']

In [9]:
big_torch_embeds = torch.Tensor(list(community_embeddings_big.values()))
small_torch_embeds = torch.Tensor(list(community_embeddings_small.values()))
pair_top10 = util.semantic_search(big_torch_embeds, small_torch_embeds, top_k=top_k)
reverse_pair_top10 = util.semantic_search(small_torch_embeds, big_torch_embeds, top_k=top_k)

In [17]:
pair_top10[221]

[{'corpus_id': 856, 'score': 0.8203004598617554},
 {'corpus_id': 106, 'score': 0.8190419673919678},
 {'corpus_id': 788, 'score': 0.8178088665008545},
 {'corpus_id': 714, 'score': 0.8137707114219666},
 {'corpus_id': 944, 'score': 0.8109714984893799},
 {'corpus_id': 761, 'score': 0.8090271949768066},
 {'corpus_id': 33, 'score': 0.8089162707328796},
 {'corpus_id': 965, 'score': 0.8070148229598999},
 {'corpus_id': 759, 'score': 0.8047701716423035},
 {'corpus_id': 418, 'score': 0.804398775100708}]

In [18]:
reverse_pair_top10[221]

[{'corpus_id': 912, 'score': 0.8845831155776978},
 {'corpus_id': 1661, 'score': 0.8809568285942078},
 {'corpus_id': 97, 'score': 0.87899249792099},
 {'corpus_id': 1665, 'score': 0.8763476014137268},
 {'corpus_id': 648, 'score': 0.8719165921211243},
 {'corpus_id': 2094, 'score': 0.8713564276695251},
 {'corpus_id': 1154, 'score': 0.8663802742958069},
 {'corpus_id': 482, 'score': 0.8649500608444214},
 {'corpus_id': 295, 'score': 0.860732913017273},
 {'corpus_id': 910, 'score': 0.859707772731781}]

In [11]:
forward_dict = dict()
backward_dict = dict()
g1_keys = list(community_embeddings_big.keys())
g2_keys = list(community_embeddings_small.keys())

for a, b in zip(g1_keys, pair_top10):
    row_info = list()
    for element in b:
        row_info.append([g2_keys[element["corpus_id"]], element["score"]])
    forward_dict[str(a)] = row_info

for a, b in zip(g2_keys, reverse_pair_top10):
    row_info = list()
    for element in b:
        row_info.append([g1_keys[element["corpus_id"]], element["score"]])
    backward_dict[str(a)] = row_info

In [14]:
with open(os.path.join(matched_communities_path, f"{big}-{small}_top_{str(top_k)}_pairs.json"), "w") as f:
    json.dump(forward_dict, f)

with open(os.path.join(matched_communities_path, f"{small}-{big}_top_{str(top_k)}_pairs.json"), "w") as f:
    json.dump(backward_dict, f)